<a href="https://colab.research.google.com/github/lasmit17/SpotifyProject/blob/master/XGBoost_Collab_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb 
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score, precision_recall_curve, auc, average_precision_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

In [3]:
luke_df_2016 = pd.read_csv(f'Luke_2016_Top_Songs.csv')
luke_df_2017 = pd.read_csv(f'Luke_2017_Top_Songs.csv')
luke_df_2018 = pd.read_csv(f'Luke_2018_Top_Songs.csv')
luke_df_2019 = pd.read_csv(f'Luke_2019_Top_Songs.csv')
luke_df_2020 = pd.read_csv(f'Luke_2020_Top_Songs.csv')

jp_df_2017 = pd.read_csv(f'jp_2017_Top_Songs.csv')
jp_df_2018 = pd.read_csv(f"jp_2018_Top_Songs.csv")
jp_df_2019 = pd.read_csv(f"jp_2019_Top_Songs.csv")
jp_df_2020 = pd.read_csv(f"jp_2020_Top_Songs.csv")

fabian_df_2018 = pd.read_csv(f"FP_2018_Top_Songs.csv")
fabian_df_2019 = pd.read_csv(f"fp_2019_Top_Songs.csv")
fabian_df_2020 = pd.read_csv(f"FP_2020_Top_Songs.csv")
fabian_df_2017 = pd.read_csv(f"FP_2017_Top_Songs.csv")

In [4]:
#Release Year
#Creating a function to add a new variable to the dataframes, the song release year
def get_years(df):
    years = []
    for date in df['release_date'].values:
        if '-' in date:
            years.append(date.split('-')[0])
        else:
            years.append(date)
    df['release_year'] = years
    return df


#Adding release year to each dataframe for each individual
luke_df_2016 = get_years(luke_df_2016)
luke_df_2017 = get_years(luke_df_2017)
luke_df_2018 = get_years(luke_df_2018)
luke_df_2019 = get_years(luke_df_2019)
luke_df_2020 = get_years(luke_df_2020)

jp_df_2017 = get_years(jp_df_2017)
jp_df_2018 = get_years(jp_df_2018)
jp_df_2019 = get_years(jp_df_2019)
jp_df_2020 = get_years(jp_df_2020)

fabian_df_2018 = get_years(fabian_df_2018)
fabian_df_2019 = get_years(fabian_df_2019)
fabian_df_2020 = get_years(fabian_df_2020)
fabian_df_2017 = get_years(fabian_df_2017)

In [5]:
#Joining DataFrames
#combining each person's data
luke_df_concat = pd.concat([luke_df_2016, luke_df_2017, luke_df_2018, luke_df_2019, luke_df_2020], ignore_index=True, axis=0)
luke_df_concat['users_name'] = "Luke"

jp_df_concat = pd.concat([jp_df_2017, jp_df_2018, jp_df_2019, jp_df_2020], ignore_index=True, axis=0)
jp_df_concat['users_name'] = "JP"

fabian_df_concat = pd.concat([fabian_df_2017, fabian_df_2018, fabian_df_2019, fabian_df_2020], ignore_index=True, axis=0)
fabian_df_concat['users_name'] = "Fabian"

#Combining every data frame together into one
all_df = pd.concat([luke_df_concat, jp_df_concat, fabian_df_concat], ignore_index=True, axis=0)

all_df["release_year"] = pd.to_numeric(all_df["release_year"])

In [6]:
#Remove Repeats
# remove repeats on individual playlists
def remove_repeats(df):
    rows_old = range(len(df['name']))
    rows_new = []
    skips = []
    for i in range(len(df['name'])):
        for j in range(i+1, len(df['name'])):
            if (df['name'][i] == df['name'][j]) and (df['artist'][i] == df['artist'][j]):
                skips.append(j)
    for row in rows_old:
        if not row in skips:
            rows_new.append(row)
    df = df.iloc[rows_new,:].reset_index(drop=True)
    return df
remove_repeats(all_df)

,name,album,artist,release_date,length,popularity,acousticness,danceability,energy,instrumentalness,...,loudness,speechiness,tempo,valence,time_signature,key,mode,uri,release_year,users_name
0,In the Bag,GO:OD AM,Mac Miller,2015-09-18,275506,52,0.3010,0.717,0.634,0.000000,...,-6.887,0.4090,143.812,0.564,4,1,1,spotify:track:7odIekt1GqLVEAAWdnd9mJ,2015,Luke
1,Gasoline,BADLANDS (Deluxe),Halsey,2015-08-28,199593,73,0.2230,0.731,0.580,0.000000,...,-7.328,0.0399,119.997,0.319,4,10,0,spotify:track:2IO7yf562c1zLzpanal1DT,2015,Luke
2,Closer,Closer,The Chainsmokers,2016-07-29,244960,85,0.4140,0.748,0.524,0.000000,...,-5.599,0.0338,95.010,0.661,4,8,1,spotify:track:7BKLCZ1jbUBVqRi2FVlTVw,2016,Luke
3,Morocco,Mountains Beaches Cities,Moon Taxi,2013-09-10,208080,0,0.0178,0.616,0.894,0.000103,...,-3.209,0.0508,115.077,0.542,4,1,0,spotify:track:30lgD1UuHczwlxa7NZFeSQ,2013,Luke
4,Temporary Love,Temporary Love,The Brinks,2015-12-04,205129,46,0.6560,0.714,0.555,0.000289,...,-7.550,0.0459,102.971,0.329,4,6,0,spotify:track:2U6hcDLZHYRSd9Up0mMe9W,2015,Luke
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1206,1975,1975,MAJAN,2019-05-10,223795,54,0.2340,0.635,0.473,0.000000,...,-10.137,0.0623,140.094,0.191,4,8,1,spotify:track:6mf2YhtRwQUdnORRrRQ3hP,2019,Fabian
1207,Whasgood?!,Whasgood?!,blackwave.,2018-06-08,221800,33,0.0311,0.851,0.669,0.000086,...,-4.259,0.0721,109.974,0.835,4,1,1,spotify:track:5vhwne4NXUgFdtbnTMz37m,2018,Fabian
1208,You Can't Hold Me Down,An Ordinary Day In An Unusual Place,Us3,2001-01-01,261466,44,0.0229,0.824,0.723,0.011100,...,-4.814,0.0305,116.013,0.963,4,1,1,spotify:track:366E1dYGmFM75DxX0yxlSq,2001,Fabian
1209,Satisfied 'N' Tickled Too,Satisfied 'N Tickled Too,Taj Mahal,1976-06-25,326826,43,0.7470,0.841,0.591,0.037100,...,-9.081,0.0527,134.796,0.817,4,3,1,spotify:track:4v6gpq8gNsvKekCc2rgXb5,1976,Fabian


In [8]:
#Scaling Variables 0<var<1
# Create function to do linear transformation on variable to change value to [0,1]
def convert_scale(df, col):
    df[col + '_old'] = df[col]
    new_max = 1
    new_min = 0
    new_range = new_max-new_min
    max_val = df[col].max()
    min_val=df[col].min()
    val_range = max_val - min_val
    df[col]=df[col].apply(lambda x: (((x-min_val)*new_range)/val_range)+new_min)
    return



#Setting the numerical spotify features
numeric_spotify_features = ['energy',
    'valence',
    'danceability',
    'liveness',
    'speechiness',
    'instrumentalness',
    'acousticness',
    'loudness',
    'length',
    'popularity',
    'tempo',
    'release_year']


for col in numeric_spotify_features:
    convert_scale(all_df, col)

In [9]:
#HotEncoding Variables
#Now we need to address the categorical variables. First, we will utilize One Hot Encoder
ohe = OneHotEncoder()
ohe_results = ohe.fit_transform(all_df[['time_signature', 'mode', 'key']])
onehot_df = pd.DataFrame(ohe_results.toarray(), columns=['time_signature_1', 'time_signature_2',
                                                     'time_signature_3', 'time_signature_4',
                                                     'mode_1', 'mode_2',
                                                     'key_1', 'key_2', 'key_3', 'key_4', 'key_5', 'key_6',
                                                     'key_7', 'key_8', 'key_9', 'key_10', 'key_11', 'key_12'])
all_df = pd.concat([all_df, onehot_df], axis=1)

In [10]:
#Combine all Features
#Finally, we combine all of our features used for prediction into one list
all_spotify_features = ['energy',
    'valence',
    'danceability',
    'liveness',
    'speechiness',
    'instrumentalness',
    'acousticness',
    'loudness',
    'length',
    'popularity',
    'tempo',
    'release_year',
    'time_signature_1',
    'time_signature_2',
    'time_signature_3',
    'time_signature_4',
    'mode_1',
    'mode_2',
    'key_1',
    'key_2',
    'key_3',
    'key_4',
    'key_5',
    'key_6',
    'key_7',
    'key_8',
    'key_9',
    'key_10',
    'key_11',
    'key_12']


PREP FINISHED

MODELS BEGIN

In [ ]:
#Now, we can finally set our X and y values. X will be all the Spotify features provided above and y will be each individual's name
X = all_df[all_spotify_features]
y = all_df['users_name']

#Here we split our data up into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=123)

